# Analyze FSF report

In [1]:
import geopandas as gpd
import pandas as pd

### Import, merge city and county reports

In [2]:
report = pd.concat([
    pd.read_csv('input/fsf-pa-county.csv').rename(columns={'County Name': 'Name'}),
    pd.read_csv('input/fsf-pa-place.csv').rename(columns={'City Name': 'Name'})
])

### Import Alleco municipalities

In [3]:
muni = gpd.read_file('input/county-muni.geojson')
muni['FIPS'] = muni.FIPS + 4200000

In [4]:
dup_muni_names = muni[muni.duplicated(subset='NAME')].NAME.unique()

muni.loc[
    (muni.NAME.isin(dup_muni_names)) & (muni.TYPE == 'BOROUGH'),
    'NAME'
] = muni.NAME + ' BR'

muni.loc[
    (muni.NAME.isin(dup_muni_names)) & (muni.TYPE == 'TOWNSHIP'),
    'NAME'
] = muni.NAME + ' TP'

### Filter report to Alleco and its municipalities

In [5]:
report = report[(report.fips == 42003) | (report.fips.isin(muni.FIPS))]

### Convert one-hour precipitation millimeters to inches

In [6]:
report['Depth_1in100_NOAA'] = report.Depth_1in100_NOAA / 25.4
report['Depth_1in100_FSF2023'] = report.Depth_1in100_FSF2023 / 25.4

### One-hour precipitation for Alleco, PGH

In [7]:
report['depth_pctchange'] = report[['Depth_1in100_NOAA', 'Depth_1in100_FSF2023']].pct_change(axis=1)['Depth_1in100_FSF2023']
report[report.fips.isin([42003, 4261000])][['Name', 'Depth_1in100_NOAA', 'Depth_1in100_FSF2023', 'depth_pctchange']]

,Name,Depth_1in100_NOAA,Depth_1in100_FSF2023,depth_pctchange
1,Allegheny,2.559055,3.574803,0.396923
1299,Pittsburgh,2.559055,3.736220,0.460000


### 1% storm frequency for Alleco, PGH

In [8]:
report['RP_1in100_NOAA'] = 100
report['100yrfreq_change'] = report.RP_1in100_NOAA / report.RP_1in100_FSF2023
report[report.fips.isin([42003, 4261000])][['Name', 'RP_1in100_NOAA', 'RP_1in100_FSF2023', '100yrfreq_change']]

,Name,RP_1in100_NOAA,RP_1in100_FSF2023,100yrfreq_change
1,Allegheny,100,25.3,3.952569
1299,Pittsburgh,100,20.3,4.926108


### 1% storm property flood risk for Alleco, PGH

In [9]:
report['100yrpropriskpct_sfha'] = report['Properties in SFHA'] / report['Total Properties']
report['100yrpropriskpct_fsf'] = report['Properties in FSF 100 Year Zone, 2023'] / report['Total Properties']
report['100yrpropriskpct_pctchange'] = report[['100yrpropriskpct_sfha', '100yrpropriskpct_fsf']].pct_change(axis=1)['100yrpropriskpct_fsf']

report['100yrpropcount_pctchange'] = report[['Properties in SFHA', 'Properties in FSF 100 Year Zone, 2023']].pct_change(axis=1)['Properties in FSF 100 Year Zone, 2023']
report[report.fips.isin([42003, 4261000])][['Name', 'Properties in SFHA', 'Properties in FSF 100 Year Zone, 2023', '100yrpropcount_pctchange', '100yrpropriskpct_pctchange']]

,Name,Properties in SFHA,"Properties in FSF 100 Year Zone, 2023",100yrpropcount_pctchange,100yrpropriskpct_pctchange
1,Allegheny,1752,85546,47.827626,47.827626
1299,Pittsburgh,216,21797,99.912037,99.912037


### 1% storm property flood risk for all Alleco municipalities

In [10]:
report[report.fips.isin(muni.FIPS)][[
    'Name',
    'Properties in SFHA',
    'Properties in FSF 100 Year Zone, 2023',
    '100yrpropriskpct_pctchange',
    '100yrpropriskpct_sfha',
    '100yrpropriskpct_fsf'
]].sort_values(by='100yrpropriskpct_fsf', ascending=False).head(10)

,Name,Properties in SFHA,"Properties in FSF 100 Year Zone, 2023",100yrpropriskpct_pctchange,100yrpropriskpct_sfha,100yrpropriskpct_fsf
1783,West Elizabeth,0,311,inf,0.000000,0.948171
1501,Sharpsburg,2,1442,720.000000,0.001281,0.923767
883,Leetsdale,1,572,571.000000,0.001304,0.745763
469,Edgeworth,0,473,inf,0.000000,0.657858
628,Glenfield,0,121,inf,0.000000,0.650538
1044,Millvale,41,1247,29.414634,0.021222,0.645445
699,Haysville,0,45,inf,0.000000,0.555556
53,Aspinwall,0,618,inf,0.000000,0.549822
954,McKees Rocks,190,1515,6.973684,0.063909,0.509586
1837,Wilmerding,0,447,inf,0.000000,0.502812


### Save Alleco municipalities list for Datawrapper

In [11]:
muni.to_file('output/county-muni.geojson', driver='GeoJSON')

### Save 1% storm property flood risk for Alleco municipalities for Datawrapper

In [12]:
pd.merge(
    report[[
        'fips',
        'Properties in SFHA',
        'Properties in FSF 100 Year Zone, 2023',
        '100yrpropriskpct_pctchange',
        '100yrpropriskpct_sfha',
        '100yrpropriskpct_fsf'
    ]],
    muni[['NAME', 'FIPS']],
    left_on='fips',
    right_on='FIPS'
).to_csv('output/county-muni-proprisk.csv', index=False)